In [4]:
import os

# Check these before running
# KPUM Workstation
studydir = ''
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# Finn's Laptop
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir
datadirbasefolder = 'derivatives/dMRI'
protocol = 'OLD' # 'NEW'

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, datadirbasefolder)
subjecttrackerfile = 'Subject_Tracker_for_dmri_pipeline.tsv'

###################################################################################
# USER INPUT

# Participant details
subject = '026'
session = 'MR1'

In [5]:
import os

# Define I/O folders and files
niftifolder = os.path.join(studydir, niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
datadir = os.path.join(studydir, datadirbasefolder,  os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)

In [ ]:
######################################
# Define local functions

def create_subjectrackertsv(subjecttrackertsv, subject, session) :   
    import os 
    import pandas as pd
    # create dataframe and write to file
    new_row = {'participant_id': [f'sub-{subject}'], 
            'session_id': [f'ses-{session}']} 
    df = pd.DataFrame(new_row)
    df.to_csv(subjecttrackertsv, sep="\t", index=False)

def add_rowtodataframe(df, subject, session) :
    import pandas as pd
    # Check if {subject} {session} has entry in DataFrame, and adds if not
    if not ((df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}')).any() :
        # Add as a new line at the bottom
        df = pd.concat([df, *[df.tail(1)]*1], ignore_index=True) # duplicates the last row in df
        # first set all columns in the new last row to ''
        df.loc[df.index[-1],:] = ''
        # then update with correct subject and session
        df.loc[df.index[-1],'participant_id'] = f'sub-{subject}'
        df.loc[df.index[-1],'session_id'] = f'ses-{session}'
    return df

def insert_process(df, process) :
    import pandas as pd
    # Check if process is in any of the df.columns
    # if not include put
    if not process in df.columns :
        # insert "process" and "process comments" columns add new two last columnsS
        df.insert(loc=len(df.columns), column=process, value='')
        df.insert(loc=len(df.columns), column=process+' comments', value='')
    return df

def checkstatus_process(df, process, subject, session, status) :
    import pandas as pd
    # Check the status of process for {subject} {session}
    outputboolean = df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), process ] == status
    return outputboolean

def setstatus_process(df, process, subject, session, status) :
    import pandas as pd
    # Check the status of process for {subject} {session}
    df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), process ] = status
    return df

def perform_process(processcall) :
    import os, subprocess
    # Perform the process given by the processcall by launching into terminal
    p=subprocess.Popen(processcall, stdout=subprocess.PIPE, shell=True)
    # Poll process.stdout to show stdout live
    while True:
        output = p.stdout.readline()
        if p.poll() is not None:
            break
        if output:
            print(output.strip().decode("utf-8"))
    rc = p.poll()

########################################
## START

import os 
import pandas as pd

currdir = os.getcwd()
os.chdir(studydir)

# Read the subjecttrackertsv-file
if not os.path.isfile(subjecttrackertsv) :
    print(f'Creating {subjecttrackertsv}')
    create_subjectrackertsv(subjecttrackertsv, subject, session)
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Check if subject session has an entry in subjecttrackertsv and add if not add one
df = add_rowtodataframe(df, subject, session)


## Process to perform - dmri_prepare_pipeline
process = 'dmri_prepare_pipeline'
processcriptfile = 'dmri_prepare_pipeline.sh'
processcriptfilepath = os.path.join(studydir,'code/kpum_noddi/shell', processcriptfile)
processcall = f"bash {processcriptfilepath} {subject} {session} -d {datadir}" 
if not process in df.columns :
    insert_process(df, process)
# IMPORTANT - Should have a check here on whether to perform, but cannot make it work 
# Decide if we are going to perform process (i.e. Status should be Pending) and perform process
#if not [ checkstatus_process(df, process, subject, session, 'Stop') ] :
#    print('We have to stop here')
#elif [ checkstatus_process(df, process, subject, session, 'Done') ] :
#    print('Process is already Done')
#else :
print(f'Status for {process} is set to Pending = need to do this step')
setstatus_process(df, process, subject, session, 'Pending')
perform_process(processcall)
setstatus_process(df, process, subject, session, 'Done')

## Process to perform - dmri_preprocess
process = 'dmri_preprocess'
processcriptfile = 'dmri_preprocess.sh'
processcriptfilepath = os.path.join(studydir,'code/kpum_noddi/shell', processcriptfile)
protocol = 'OLD' # or 'NEW'
processcall = f"bash {processcriptfilepath} {subject} {session} -d {datadir} -protocol {protocol}" 
if not process in df.columns :
    insert_process(df, process)
# IMPORTANT - Should have a check here on whether to perform, but cannot make it work 
# Decide if we are going to perform process (i.e. Status should be Pending) and perform process
#if not [ checkstatus_process(df, process, subject, session, 'Stop') ] :
#    print('We have to stop here')
#elif [ checkstatus_process(df, process, subject, session, 'Done') ] :
#    print('Process is already Done')
#else :
print(f'Status for {process} is set to Pending = need to do this step')
setstatus_process(df, process, subject, session, 'Pending')
perform_process(processcall)
setstatus_process(df, process, subject, session, 'Done')

os.chdir(currdir)
